# Analysis of the results

Here we analyse the EER for the three ASV systems and both attack scenarios

In [1]:
import os
import numpy as np

In [2]:
# define the dirs that will be included in this analysis
log_dirs = {
    "ecapa": ["../../logs/stargan/asv_ecapa", "../../logs/stargan/asv_ecapa_2", "../../logs/stargan/asv_ecapa_3"],
    "ecapa-lda": ["../../logs/stargan/asv_ecapa_lda100", "../../logs/stargan/asv_ecapa_lda100_2", "../../logs/stargan/asv_ecapa_lda100_3"],
    "xvect-lda": ["../../logs/stargan/asv_xvect_lda200", "../../logs/stargan/asv_xvect_lda200_2", "../../logs/stargan/asv_xvect_lda200_3"],
    "concat-lda": ["../../logs/stargan/asv_concat_lda200", "../../logs/stargan/asv_concat_lda200_2", "../../logs/stargan/asv_concat_lda200_3"],
}

In [3]:
components = {
    "ignorant": "eval/asv-plda/ignorant/results/eer.txt",
    "lazy-informed": "eval/asv-plda/lazy-informed/results/eer.txt",
}

In [4]:
# aggregate results by the keys of the target and source speakers
results = dict()
for asv_config in log_dirs:

    results[asv_config] = dict()
    for log_dir in log_dirs[asv_config]:
        for component, eval_file in components.items():

            path = os.path.join(log_dir, eval_file)
            with open(path, "r") as f:
                next(f)  # skip header

                for line in f:
                    # get the value for the given dataset and key to the results
                    elements = line.split()
                    dataset = elements[0]
                    value = elements[-1]
                    if dataset.endswith(".txt"):
                        dataset = dataset[:-4]
                    # add the dataset to the results if needed
                    if dataset not in results[asv_config]:
                        results[asv_config][dataset] = dict()
                    # add the component to the results if needed
                    if component not in results[asv_config][dataset]:
                        results[asv_config][dataset][component] = list()
            
                    results[asv_config][dataset][component].append(float(value))

In [5]:
# average the results of the different components
avgs = dict()
for asv_config in results:
    avgs[asv_config] = dict()
    for dataset in results[asv_config]:
        avgs[asv_config][dataset] = dict()
        for component in results[asv_config][dataset]:
            avgs[asv_config][dataset][component] = sum(results[asv_config][dataset][component]) / len(results[asv_config][dataset][component])

In [6]:
# print the results in a markdown table
print("| Dataset |", end=" ")  # empty cell in the top left corner
# print the config names as column headers
for asv_config in avgs:
    print(asv_config, end=" | ")
print()  # new line

# add hyphens to separate the header from the table
print("|", end=" ")
for i in range(len(avgs["ecapa"]) + 1):
    print("---", end=" | ")
print()  # new line

# print the averages of each dataset for each ASV config
for dataset in avgs["ecapa"]:
    print("|", end=" ")
    print(dataset, end=" | ")
    for asv_config in avgs:
        print(" / ".join([str(round(avg*100, 2)) for avg in avgs[asv_config][dataset].values()]), end=" | ")
    print()  # new line

# prnt the averages of each asv config
print("| Average |", end=" ")
for asv_config in avgs:
    total_sum = np.zeros(2)
    for dataset in avgs[asv_config]:
        total_sum += np.array(list(avgs[asv_config][dataset].values()))
    dataset_avgs = total_sum / len(avgs[asv_config])
    print(" / ".join([str(round(avg*100, 2)) for avg in dataset_avgs]), end=" | ")
print()  # new line


| Dataset | ecapa | ecapa-lda | xvect-lda | concat-lda | 
| --- | --- | --- | --- | --- | 
| cv-test_3utts | 34.18 / 33.26 | 36.49 / 33.4 | 27.22 / 21.68 | 33.51 / 21.97 | 
| ls-test-clean | 32.18 / 36.52 | 34.34 / 36.67 | 32.46 / 26.77 | 38.8 / 26.18 | 
| edacc-test | 39.91 / 41.43 | 39.8 / 41.17 | 37.89 / 28.4 | 39.99 / 30.15 | 
| ravdess | 43.42 / 43.84 | 46.8 / 42.75 | 43.66 / 35.81 | 46.62 / 34.48 | 
| Average | 37.42 / 38.76 | 39.36 / 38.5 | 35.31 / 28.17 | 39.73 / 28.19 | 


### EERs for each ASV config

| Dataset | ecapa | ecapa-lda | xvect-lda | concat-lda | 
| --- | --- | --- | --- | --- | 
| cv-test_3utts | 34.18 / 33.26 | 36.49 / 33.4 | 27.22 / 21.68 | 33.51 / 21.97 | 
| ls-test-clean | 32.18 / 36.52 | 34.34 / 36.67 | 32.46 / 26.77 | 38.8 / 26.18 | 
| edacc-test | 39.91 / 41.43 | 39.8 / 41.17 | 37.89 / 28.4 | 39.99 / 30.15 | 
| ravdess | 43.42 / 43.84 | 46.8 / 42.75 | 43.66 / 35.81 | 46.62 / 34.48 | 
| Average | 37.42 / 38.76 | 39.36 / 38.5 | 35.31 / 28.17 | 39.73 / 28.19 | 